# User_antenna mapping para argentina
La idea es usar el el raw CDRs dataset para crear la lista que cuenta antennas utilizadas por cada user diariamente y agrupar esta lista (con repeticiones) en un agregado **DE LOS 5 MESES** (para luego determinar home_antenna etc.)

Tambien usando info de las antenas (epidemicas o no). Podriamos ver si un user "vive" o solo "paso" por la zona epidemica.

## definiciones utilizadas

* la fecha viene dado en YYYMMDD y el TIME es HHMMSS en formato 24 horas
* *Time* columns are measured in seconds.
* Para establecer home antenna **Caro** define el daylight como de [5 20) (redondedo en segundos) timeframe (using Arg's timezone)
* Para establecer la diferencia work/no_work **Caro** define el work_time como el [9 19) (redondeado en segundos) timeframe (using Arg's timezone). Esta diferencia se utiliza para work/non_work users/calls.

* el analisis es solo para CDRs con usuarios de la TelCo. Los users de la TelCo van todos en la primer columna (independientemente del nombre de la columna..) entonces si la llamada es entrante quiere decir que el user de la telco esta recibiendo un llamado en esa antennaID. si la llamada es saliente es al reves. (Podria pasar que sea comunicaciones inter-Telco y ahi habria dos records uno saliente y otro entrante pero con los mismos 2 userIDs).
* La Direction, viene dada relativa al user de la primer columna (incoming ,1, si entra y viceversa si es outgoing,0).

### el dataset de crudo vendria con este header
ORIGIN_NUMBER_ENC_B64|TARGET_NUMBER_ENC_B64|DIRECTION|CLIENT_BUSSINES_ID|OTHER_OPERATOR|FECHA|TIME|DURATION|CLIENT_CELL_ID|REDIRECT_VOICEMAIL|AIR_CHARGE|LAND_CHARGE|MONEDERO|RATE_ID_AIR|RATE_ID_LAND|DISCOUNT_AMOUNT_FF

Basicamente los primeros 2 son los hashes de los users que estuvieron en el call, con el primero el user de la telco.

Direction es quien llama a quien (outgoing es 1-->2, viceversa incoming). Fecha el dia,mes,anyo del llamado, time en formato 24hs incluyendo segundos. Duration medido en segundos, Client_cell_id el hash de la antenna y todo el resto no nos interesa.

## Differences with Caro's processing

First, datasets used are different, were she uses already surrogated files, with timestamps in seconds since epoch, antennas as numbers (not hashes) and 

In [1]:
%autosave 180
import numpy as np; import os;import random;
import graphlab as gl
#esto es para dibujar directo a la notebook
gl.canvas.set_target('ipynb')

#from matplotlib import pyplot as plt
#%matplotlib inline
import time
import os
np.random.seed(2016)

Autosaving every 180 seconds


/home/juan/mobility-study/scikit-learn/sklearn/externals/joblib/_multiprocessing_helpers.py:29: UserWarning: [Errno 38] Function not implemented.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))
A newer version of GraphLab Create (v1.10.1) is available! Your current version is v1.8.5.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [2]:
#ejemplo  "/grandata/simple_format/01-02012014.txt.gz"
#seteamos el lugar de trabajo
rootdir="/grandata/ca/voice/"
os.chdir(rootdir)
year = 2011; 
months= [1,2,3];
days = range(1,30)

def get_raw_file(month,day):
    return rootdir +"{y}/{m}/binaria_gsm_{y}{m:0=2d}{d:0=2d}.csv.gz"\
                .format(y=2012,m=month,d=day)
    
def get_output_file(group = -1,sample=False):
    output = "/home/juan/mobility-study/argentina-scripts/output/homeantenna_sframe"
    if sample == True:
        output = output + "_sample"
    if group != -1:
        output = output + "_group{g}_1".format(g=group)
    return output


In [3]:
days = range(1,30)
month = 3
day = days[10]
raw_file = get_raw_file(month,day)

In [4]:
!zcat $raw_file | wc -l
!zcat $raw_file | head

44858093
ORIGIN_NUMBER_ENC_B64|TARGET_NUMBER_ENC_B64|DIRECTION|CLIENT_BUSSINES_ID|OTHER_OPERATOR|FECHA|TIME|DURATION|CLIENT_CELL_ID|REDIRECT_VOICEMAIL|AIR_CHARGE|LAND_CHARGE|MONEDERO|RATE_ID_AIR|RATE_ID_LAND|DISCOUNT_AMOUNT_FF
4753724553746A6E56686D5849536577436B375550513D3D|4C4434427355663273354B4D78492B4168754D5457673D3D|O|CO|CTI PCS S.A.|20120311|125215|195|RN203B|N|0|0|GRANEL|ALOCAL|CLV09|0
307A344967764170452B6A75626E433538424D6743513D3D|6B6C6F483737453548515A5048595A54656F535177773D3D|O|CR|CTI PCS S.A.|20120311|223605|170|071|N|173959|34969|BONUS|||0
7A4F636562387674592B526B684E32396972524D78773D3D|576B34494170732B327778787A447A47766C764378413D3D|O|CO|COMPA?IA DE RADIOCOMUNICACIONES MOVILES S.A.|20120311|120615|133|C1089J|N|.99|0|GRANEL|ALOCAL|TFIJO|0
4F43594A33334473504E39554D526641564B65634B513D3D|6B4633425842726E49497867307A4F74547353322F513D3D|O|CR|NEXTEL COMMUNICATIONS ARGENTINA S.A.|20120311|190253|7|CTI 1|N|88988|17888|PERIODIC|||0
4A6C66694B6D525A6D6C7056546F5A55376634697

## Analytics on files
Hay masomenos 40~60 de lineas x dia, i.e. 40 a 60m de llamados

## Probamos con GraphLab gl
tiene buenas conexiones con Spark, es mas rapido para los applys (ya probado) y parece ser mejor en terminos de que puede levantar files mas grandes que el tamanyo que tienen en la memoria. 

Ademas nativamente paraleliza todo con cython. Mirar htop como impacta cuando usamos 'applys' sobre el dataset.

En gl pesa 3GB lo que con pd pesaba 10GB!!

In [5]:
antennas_file = '/home/juan/mobility-study/argentina-scripts/data/celdas_limpio.csv'
antennas = gl.SFrame.read_csv( antennas_file, delimiter='|', 
                                     header=True, skip_initial_space=True, 
                                     column_type_hints = [str,float, float, str, str, bool], 
                                     na_values=['NaN'],
                                     usecols = ['CEL_ID','LATITUD',
                                                'LONGITUD','DEPARTAMENTO','PROVINCIA','EPIDEMIC'],  
                                     error_bad_lines=False,
                                     verbose = False
                                    )                

This non-commercial license of GraphLab Create is assigned to svega@dm.uba.ar and will expire on April 16, 2017. For commercial licensing options, visit https://dato.com/buy/.


2016-06-24 13:58:25,230 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1466776703.log


In [7]:
%%time
import datetime

#columnt_type_hints=[str, str, str, str, str,int ,int ,int ,
#                                      str, str, float,float,str,str,str,int ] 

#obs. NO levanto la columna Fecha pues esta info la puedo sacar del filename
current_date = datetime.datetime(2012,month,day)

test = gl.SFrame.read_csv( raw_file, delimiter='|', 
        header=True, skip_initial_space=True, 
        column_type_hints = [str,str, str,str, str, str, int], 
        #nrows = 25*1e6,
        usecols = ['ORIGIN_NUMBER_ENC_B64',"TARGET_NUMBER_ENC_B64",'DIRECTION',
                                                    'FECHA','TIME','CLIENT_CELL_ID','DURATION'],  
        error_bad_lines=False,
                  )

test.rename({'ORIGIN_NUMBER_ENC_B64':'USER',
                      'FECHA':'DATE'})



Read 280013 lines. Lines per second: 55732.2

Read 1122780 lines. Lines per second: 107539

Read 2215616 lines. Lines per second: 134075

Read 3276132 lines. Lines per second: 143452

Read 4161796 lines. Lines per second: 145413

...

Read 43635006 lines. Lines per second: 154763

Read 44545872 lines. Lines per second: 154228

Finished parsing file /grandata/ca/voice/2012/3/binaria_gsm_20120311.csv.gz

Parsing completed. Parsed 44858092 lines in 289.715 secs.

CPU times: user 11min 14s, sys: 38.8 s, total: 11min 53s
Wall time: 4min 52s


In [8]:
test.shape

(44858092, 7)

In [9]:
%%time
test['CLIENT_CELL_ID'] = test['CLIENT_CELL_ID'].apply(lambda x: x[:-1] if len(x) ==6 else x)
test = test.filter_by(antennas['CEL_ID'],'CLIENT_CELL_ID')

CPU times: user 8min 57s, sys: 1min 5s, total: 10min 3s
Wall time: 2min 42s


In [10]:
test.shape

(21192261, 7)

In [22]:
test.sample(0.0001)

USER,TARGET_NUMBER_ENC_B64,DIRECTION,DATE,TIME,CLIENT_CELL_ID,DURATION
586C657674677276587268734F6D3231384646435A413D3D ...,51464731484B41796C30576E43735A556848425577413D3D ...,O,20120311,020445,SF026,6
6248436242434854473476736A4D6C41524E644E71773D3D ...,6B7737334E4B4249616677787A364B324357507931413D3D ...,O,20120311,182131,CB224,7
62626466364A32524A765972457279353762596C49673D3D ...,4A7571506F4A4D36485555354F316F377A6D654732673D3D ...,O,20120311,011800,CF146,53
746778454E5679756E5431356569357A3762355776773D3D ...,3271544874375146456E615656686A565252654D43413D3D ...,O,20120311,190928,ME050,1855
52613132557354633834727577546F3971366E3149413D3D ...,6B75412F43654F45793138664D5257316A692F7578413D3D ...,O,20120311,175206,ME092,2
52675130414E58506E51502B686F48596E72513379513D3D ...,6364324A445578744335486E7A4F79756A584B366A773D3D ...,O,20120311,175659,BA398,16
6E3978532B6C516F667251736C72704974495A5070513D3D ...,4F31646C2F396236634E66514E6448775352454C42673D3D ...,O,20120311,221337,C1872,125
51756D364E447472795A7855314F37577649726351773D3D ...,757A374B6E4135353372497173575853465752486C673D3D ...,O,20120311,133825,ER146,159
2F4E36334B6A4B646749457577334236672B356337673D3D ...,2F4E36334B6A4B6467494771673039623933794F31513D3D ...,O,20120311,134918,SF380,462
75775A424D616F30593370724755636963556B6E52673D3D ...,474D7955327A36536641585670734135592F586662513D3D ...,O,20120311,134033,CO664,8


In [6]:
test.head()

USER,TARGET_NUMBER_ENC_B64,DIRECTION,CLIENT_BUSSINES_ID,OTHER_OPERATOR
4753724553746A6E56686D5849536577436B375550513D3D ...,4C4434427355663273354B4D78492B4168754D5457673D3D ...,O,CO,CTI PCS S.A.
307A344967764170452B6A75626E433538424D6743513D3D ...,6B6C6F483737453548515A5048595A54656F535177773D3D ...,O,CR,CTI PCS S.A.
7A4F636562387674592B526B684E32396972524D78773D3D ...,576B34494170732B327778787A447A47766C764378413D3D ...,O,CO,COMPA?IA DERADIOCOMUNICACIONES ...
4F43594A33334473504E39554D526641564B65634B513D3D ...,6B4633425842726E49497867307A4F74547353322F513D3D ...,O,CR,NEXTEL COMMUNICATIONSARGENTINA S.A. ...
4A6C66694B6D525A6D6C7056546F5A553766346972773D3D ...,454B4C562B4245444C614D4B4C6B6D554E4639706B773D3D ...,O,CR,GTE PCS S.A.
50356C3576502F34482B4239337550524A634E4C4C413D3D ...,71692F55737A446873726568413949344865727252413D3D ...,O,CR,COMPA?IA DERADIOCOMUNICACIONES ...
50356C3576502F34482B4239337550524A634E4C4C413D3D ...,71692F55737A446873726568413949344865727252413D3D ...,O,CR,COMPA?IA DERADIOCOMUNICACIONES ...
7A3562492F7A68317A7244597147774752532F6B6F673D3D ...,446A7477456E725A58337937484E64546C675A7641413D3D ...,O,CO,CTI PCS S.A.
4D4F6E43546656686D335350624A5469334574564B673D3D ...,72325A4B3468663745346C75315831666930433261673D3D ...,O,CO,CTI PCS S.A.
523853547762703631654B425344644C514F37574E673D3D ...,512F7867357A52394B7539513277755639366B4149413D3D ...,O,CO,CTI PCS S.A.


In [7]:
%time
uniques = dict()
for col in ['DIRECTION','CLIENT_BUSSINES_ID','OTHER_OPERATOR','CLIENT_CELL_ID','RATE_ID_LAND']:
    vals = test[col].unique().to_numpy()
    length = len(vals)
    uniques[col] = {'length':length,'values':vals}
    print('Col {c} has {n} unique values'.format(c = col, n = length))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 17.9 µs
Col DIRECTION has 2 unique values
Col CLIENT_BUSSINES_ID has 4 unique values
Col OTHER_OPERATOR has 385 unique values
Col CLIENT_CELL_ID has 25685 unique values
Col RATE_ID_LAND has 15 unique values


In [164]:
test[test['CLIENT_CELL_ID'].apply(lambda x: True if 'CTI ' in x else False)!=1]

USER,TARGET_NUMBER_ENC_B64,DIRECTION,CLIENT_BUSSINES_ID,OTHER_OPERATOR
4753724553746A6E56686D5849536577436B375550513D3D ...,4C4434427355663273354B4D78492B4168754D5457673D3D ...,O,CO,CTI PCS S.A.
307A344967764170452B6A75626E433538424D6743513D3D ...,6B6C6F483737453548515A5048595A54656F535177773D3D ...,O,CR,CTI PCS S.A.
7A4F636562387674592B526B684E32396972524D78773D3D ...,576B34494170732B327778787A447A47766C764378413D3D ...,O,CO,COMPA?IA DERADIOCOMUNICACIONES ...
7A3562492F7A68317A7244597147774752532F6B6F673D3D ...,446A7477456E725A58337937484E64546C675A7641413D3D ...,O,CO,CTI PCS S.A.
4D4F6E43546656686D335350624A5469334574564B673D3D ...,72325A4B3468663745346C75315831666930433261673D3D ...,O,CO,CTI PCS S.A.
523853547762703631654B425344644C514F37574E673D3D ...,512F7867357A52394B7539513277755639366B4149413D3D ...,O,CO,CTI PCS S.A.
52754A4B746D77556F512F47584E376143384A4857673D3D ...,3641794746396D71564B6A536938483534672F3647673D3D ...,O,CO,CTI PCS S.A.
6B626C46334832616E4643556259766B3736707776673D3D ...,4D6E6E644D3644737249324279515242766B6A3530673D3D ...,O,CO,TELECENTRO S.A.
577A535278494644494E712B627436692B687A367A513D3D ...,6C375063313876754E7745626E682F573176733846673D3D ...,O,CR,TELECOM PERSONAL S.A.
474937784D505452657A59684349776A6A41655156513D3D ...,696A6B527A584E4953353830664E55773255656F70773D3D ...,O,CR,TELECOM PERSONAL S.A.


In [13]:
antennas.head()

CEL_ID,LATITUD,LONGITUD,DEPARTAMENTO,PROVINCIA,EPIDEMIC
B1061,-36.339,-56.7468,La_Costa,Buenos_Aires,0
B1062,-33.4578,-60.0522,Ramallo,Buenos_Aires,0
BA001,-35.1641,-60.0866,Veinticinco_de_Mayo,Buenos_Aires,0
BA003,-35.4275,-60.1538,Veinticinco_de_Mayo,Buenos_Aires,0
BA004,-35.9,-60.7275,Veinticinco_de_Mayo,Buenos_Aires,0
BA006,-35.2355,-60.8283,Nueve_de_Julio,Buenos_Aires,0
BA007,-35.2894,-61.3972,Nueve_de_Julio,Buenos_Aires,0
BA010,-35.6458,-60.7036,Nueve_de_Julio,Buenos_Aires,0
BA013,-37.1577,-63.2363,Adolfo_Alsina,Buenos_Aires,0
BA015,-37.9641,-60.4202,Adolfo_Gonzales_Chaves,Buenos_Aires,0


In [11]:
test.shape[0]/1e6

21.192261

## SFrame MAGIC!
Se pueden guardar en disco los sframes con un formato especial para que despues sean leidos 'casi' instantaneamente

In [12]:
%%time
## TESTEO que tanto cuesta guardar un sframe con 10M de lineas a disco! pues resulta mas conveniente para despues cargar 
#esta info nuevamente cuando la necesite
sample_save_dir = '/home/juan/mobility-study/argentina-scripts/test_sframe_in_disk'
test.save(sample_save_dir)

CPU times: user 14.5 s, sys: 1.39 s, total: 15.8 s
Wall time: 19.8 s


In [17]:
del test

In [19]:
!du -sh $sample_save_dir
!du -sh $raw_file
!du -sh "/home/juan/mobility-study/argentina-scripts/sframe_cdrs/2012/"

1.2G	/home/juan/mobility-study/argentina-scripts/test_sframe_in_disk
1.8G	/grandata/ca/voice/2012/3/binaria_gsm_20120311.csv.gz
317G	/home/juan/mobility-study/argentina-scripts/sframe_cdrs/2012/


In [18]:
%%time
test = gl.load_sframe('/home/juan/mobility-study/argentina-scripts/test_sframe_in_disk')

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 3.7 ms


In [35]:
test.head()

USER,OTHER_USER,DIRECTION,DATE,TIME,CLIENT_CELL_ID
703344566D793559654176393030796754677A4F45513D3D ...,3861696F2B5634476F634844504C52593768314E50673D3D ...,O,20111101,120814,CF001G
4D6A766A51747A5276793748392B7964426374776D513D3D ...,49385033304F594B5941336F644A7350534B49724A413D3D ...,O,20111101,201546,CTI 1
4A6C66694B6D525A6D6C7056546F5A553766346972773D3D ...,454B4C562B4245444C614D4B4C6B6D554E4639706B773D3D ...,O,20111101,220049,CTI 1
314234775A654162706B4346556C614974334F396F673D3D ...,38422F55654542727A564B686C7935305875594953673D3D ...,O,20111101,195837,C1066I
314234775A654162706B4346556C614974334F396F673D3D ...,38422F55654542727A564B686C7935305875594953673D3D ...,O,20111101,195526,C1066Z
7A3562492F7A68317A7244597147774752532F6B6F673D3D ...,446A7477456E725A58337937484E64546C675A7641413D3D ...,O,20111101,194631,SE088G
7A3562492F7A68317A7244597147774752532F6B6F673D3D ...,446A7477456E725A58337937484E64546C675A7641413D3D ...,O,20111101,114929,SE088G
7A3562492F7A68317A7244597147774752532F6B6F673D3D ...,446A7477456E725A58337937484E64546C675A7641413D3D ...,O,20111101,110830,SE088G
5565437763513268337171784C754649724C306B31673D3D ...,674B46417734622F694A352F69632F695256303256773D3D ...,O,20111101,111841,BA098B
4C616164644E2B4337676A5462456D4C4179555253413D3D ...,396566426F525352424B6E6B3168575833534B4F70413D3D ...,O,20111101,132618,CTI 1


## Cargo el file limpio
de antennas para filtrar solo por aquellas que nos interesan

In [9]:
antennas_file = '/home/juan/mobility-study/argentina-scripts/data/celdas_limpio.csv'
antennas = gl.SFrame.read_csv( antennas_file, delimiter='|', 
                                         header=True, skip_initial_space=True, 
                                         column_type_hints = [str,float, float, str, str, bool], 
                                         na_values=['NaN'],
                                         usecols = ['CEL_ID','LATITUD',
                                                    'LONGITUD','DEPARTAMENTO','PROVINCIA','EPIDEMIC'],  
                                         error_bad_lines=False,
                                         verbose = False
                                        )
!head $antennas_file

CEL_ID|CEL_DESCRIPTION|LATITUD|LONGITUD|DEPARTAMENTO|PROVINCIA|GC_AREA|EPIDEMIC
B1061|MUNDO_MARNO|-36.339|-56.7468|La_Costa|Buenos_Aires|NOT_GC|0
B1062|BUNGE_RAMALLO|-33.4578|-60.0522|Ramallo|Buenos_Aires|NOT_GC|0
BA001|GOBERNADOR_UGARTE|-35.1641|-60.0866|Veinticinco_de_Mayo|Buenos_Aires|NOT_GC|0
BA003|25_DE_MAYO|-35.4275|-60.1538|Veinticinco_de_Mayo|Buenos_Aires|NOT_GC|0
BA004|DEL_VALLE|-35.9|-60.7275|Veinticinco_de_Mayo|Buenos_Aires|NOT_GC|0
BA006|CARLOS_M_NAON|-35.2355|-60.8283|Nueve_de_Julio|Buenos_Aires|NOT_GC|0
BA007|ADEMARCHI_(ESTQUIR|-35.2894|-61.3972|Nueve_de_Julio|Buenos_Aires|NOT_GC|0
BA010|DUDIGNAC|-35.6458|-60.7036|Nueve_de_Julio|Buenos_Aires|NOT_GC|0
BA013|RIVERA|-37.1577|-63.2363|Adolfo_Alsina|Buenos_Aires|NOT_GC|0


In [28]:
import pandas as pd; import numpy as np; import os;import random;
import graphlab as gl
pd.set_option('display.max_rows', 200)
#esto es para dibujar directo a la notebook
gl.canvas.set_target('ipynb')

#from matplotlib import pyplot as plt
#%matplotlib inline
import time
import os
import datetime
np.random.seed(2016)
import subprocess

#seteamos el lugar de trabajo
rootdir="/home/juan/mobility-study/argentina-scripts"
os.chdir(rootdir)

def get_input_file(year,month,day):
    return rootdir +"{y}/{m}/binaria_gsm_{y}{m:0=2d}{d:0=2d}.csv.gz"\
                .format(y=year,m=month,d=day)

def get_output_file(group = -1,sample=False):
    output = "/home/juan/mobility-study/argentina-scripts/output/user_antenna_mapping_sframe"
    if sample == True:
        output = output + "_sample_group{g}_1".format(g=group)
    if group != -1:
        output = output + "_group{g}_1".format(g=group)
    return output

#from dictionary output the order list of most_used_antennas with their corresponding counts
def get_columns_from_dict(user_dict):
    antenna_count_list = [-1]*10
    #por un problema de type consistency dentro de la listas que le meto al SFrame, aca los -1 tienen que ser strings
    antenna_list = ["-1"]*10
    sorted_user_dict = sorted(user_dict, key=user_dict.get, reverse=True)
    for i,ant in enumerate(sorted_user_dict):
        if(i>9):
            break
        antenna_count_list[i] = user_dict[ant]
        antenna_list[i] =  ant
            #print(user_dict)
        #for other than the first case
        if i==0:
            continue
        #random permutation of current and previous antenna in case of a count tie
        if user_dict[sorted_user_dict[i-1]]==user_dict[ant]  and np.random.random() <= 0.5:
            #add past antennas data to current
            antenna_count_list[i] = antenna_count_list[i-1]
            antenna_list[i] =  antenna_list[i-1]
            ## add current data to past
            antenna_count_list[i-1] = user_dict[ant]
            antenna_list[i-1] =  ant
    
    rv = (antenna_list, antenna_count_list)
    return rv

def printUsage():
    print "Usage:"
    print "python userAntennaMapping.py [mode]"
    print "mode = 'true' to test out the algorithm on a sample of upto 7 days for a (random) particular month"
    print "mode = 'false' to run it for all days/months"


In [29]:
test = True

#para leer solo un 10% de c/ dataset e ir operando ahi.
if test:
    nrows = 6*1e6

In [30]:
start_time = time.time()


#generamos el diccionario que setea los parametros sobre el cual vamos a iterar
year = 2012
ms = [11,12,1,2,3]
if test:
    ms = list(np.random.choice(ms,(1,)))
    print('test month is %s' %str(ms))

months = dict()

for month in ms:
        days = range(1,32)
        #check november days length
        if month ==11:
            days = range(1,31)
        #check february days length
        if month ==2:
            days = range(1,30)
        #alterno el year para los meses que arrancan en enero
        if month > 10:
            year = 2011
        #si es test elijo una tira de dias de random_length h/7
        if test:
            length = np.random.choice(range(1,8),(1,))[0]
            days = np.random.choice(range(1,30),(length,),replace=False)

        months[month] = {"year":year,"days":days}
        #print(month,year)
if test:
    print('we have this month {m} and these days {ds}'.format(m = month,ds = str(days)))

## we first check all the necessary sframe datasetes exist
for month in months:
    year = months[month]['year']
    days = months[month]['days']
    for day in days:
        sframe_dir = '/home/juan/mobility-study/argentina-scripts/sframe_cdrs/{y}/{m:0=2d}/{d:0=2d}'.format(y=year,m=month,d=day)
        if not(os.path.exists(sframe_dir)):
            print("Unexistent sframe for {y}/{m:0=2d}/{d:0=2d}. Stopping the algorithm, get sframes".format(y = year, m = month,d=day) ) 

test month is [2]
we have this month 2 and these days [28 18  1 27 17 19  6]


In [104]:
table.head()

USER,OTHER_USER,DIRECTION,CLIENT_CELL_ID,TIMESTAMP,DURING_WEEKEND
783258682B392B3337655836314C4343784A345332773D3D ...,783258682B392B3337655836314C4343784A345332773D3D ...,O,CTI 1,2011-12-26 21:50:24,0
4D4D344E6C42552B613272715254596C332F354F69413D3D ...,33706576324147722B753047616A31762F55642B37773D3D ...,O,CTI 1,2011-12-26 18:24:13,0
4E6E4E316877754C7946374A51734473684869676C413D3D ...,645373534771415039344E5849694163654D365261413D3D ...,O,CTI 1,2011-12-26 15:31:02,0
78554A4D52544E3070346E4A6F474A66434D6A6753513D3D ...,554B304B6669625669626A58614F6D382F7A4A564E413D3D ...,O,CTI 1,2011-12-26 15:03:26,0
4E74774A324A50633977436564336A597847654F30413D3D ...,417A4B743949696144536750394850467139455476673D3D ...,O,CTI 1,2011-12-26 20:21:36,0
7934544646773439324F32753644734B2F376D3332773D3D ...,737A622F506364495A41395230475A68754137756B513D3D ...,O,CTI 1,2011-12-26 21:11:49,0
446C6B4F432F542B6964765252726C30466B43736D773D3D ...,4C585258386350774F6E625A5A4F43425339374C4E413D3D ...,O,CTI 1,2011-12-26 07:30:48,0
446C6B4F432F542B6964765252726C30466B43736D773D3D ...,4C585258386350774F6E625A5A4F43425339374C4E413D3D ...,O,CTI 1,2011-12-26 00:35:46,0
4A696A61546874736430766544576A624C31584B74773D3D ...,347A474A36467748316E6B44762B3168716F41594E513D3D ...,O,CTI 4,2011-12-26 21:08:36,0
52395154774F6537706C6E6C536377675262513675513D3D ...,6D345261664C4D4A7955537267524E49426247734F673D3D ...,O,CTI 1,2011-12-26 22:40:39,0


In [31]:
local_time = time.localtime()

print("Read all Sframe dirs and append to one big table, time elapsed is {t}  localtime is {ho}:{mi:0=2d}".format(t=(time.time()-start_time), ho = local_time.tm_hour , mi = local_time.tm_min ))

#aca seteamos como vamos a partir la tabla segun el nro correspondiente a c/hash y tomando modulo
passes = 10 

#creo los grupos que despues van a filtrar c/chunk de la tabla para hacer varias pasadas
group = 3

local_time = time.localtime()
print("working group number {it} of {pas}, time elapsed is {t} \n".format(it=group,pas=passes, t=(time.time()-start_time)))
#itero sobre todos los day-months
table =  gl.SFrame()

for month in months:
    year = months[month]['year']
    days = months[month]['days']
    print(year,month,str(days))
    for day in days:
        #si estoy "reiniciando" el algoritmo, chequeo que el output no exista ya
        output_file = get_output_file(group=group)
        #si es un test ni hago el check
        if not(test) and (os.path.exists(output_file)):
            print("skipping {m:0=2d}/{d:0=2d} since output_dir exists".format(m=month,d=day) ) 
            continue

        sframe_dir = '/home/juan/mobility-study/argentina-scripts/sframe_cdrs/{y}/{m:0=2d}/{d:0=2d}'.format(y=year,m=month,d=day)

        local_time = time.localtime()
        print("Reading sframe file for day {d}-{ms}, time elapsed is {t} localtime is {ho}:{mi:0=2d} ".\
              format(d=day,ms=month,t=(time.time()-start_time),ho=local_time.tm_hour,mi=local_time.tm_min))

        daily_table = gl.load_sframe(sframe_dir)
        daily_table.remove_columns(['OTHER_OPERATOR','DURATION'])
        ## por el problema de las antenas invisibles tipo 'CTI 1', 'CTI 2', etc. voy a quedarme con las filas del CDR para 
        # el cual se exactamente el lat/long de c/ antenna.
        daily_table['CLIENT_CELL_ID'] = daily_table['CLIENT_CELL_ID'].apply(lambda x: x[:-1] if len(x) ==6 else x)
        daily_table = daily_table.filter_by(antennas['CEL_ID'],'CLIENT_CELL_ID')

                
        #obs. NO levanto la columna Fecha pues esta info la puedo sacar del filename
        #current_date = datetime.datetime(year,month,day)

        if test:
            daily_table = daily_table.head(nrows)
        #a cada tabla diaria filtro por todos los USERs por el hash del string (el hash es un int) y despue filtro modulo passes
        #y trabajo sobre la tabla subgroup nada mas que ahora tiene menos usuarios
        table = table.append(daily_table[daily_table['USER'].apply(lambda x: hash(x) % passes == group )])

        del daily_table

    #entonces la idea es que yo ahora solo voy a trabajar, dentro de esta tabla filtrada y para todos los dias del mes juntos

print('finished day reading for group {g} of {p}, time elapsed is {t} localtime is {ho}:{mi:0=2d} '.format(g = group,
                                    p = passes, t=(time.time()-start_time), ho=local_time.tm_hour,mi=local_time.tm_min))

Read all Sframe dirs and append to one big table, time elapsed is 17.4628210068  localtime is 3:02
working group number 3 of 10, time elapsed is 17.4632840157 

(2012, 2, '[28 18  1 27 17 19  6]')
Reading sframe file for day 28-2, time elapsed is 17.4684581757 localtime is 3:02 
Reading sframe file for day 18-2, time elapsed is 219.267994165 localtime is 3:06 
Reading sframe file for day 1-2, time elapsed is 390.746299028 localtime is 3:09 
Reading sframe file for day 27-2, time elapsed is 583.23221302 localtime is 3:12 
Reading sframe file for day 17-2, time elapsed is 726.121207952 localtime is 3:14 
Reading sframe file for day 19-2, time elapsed is 911.606332064 localtime is 3:17 
Reading sframe file for day 6-2, time elapsed is 1042.69546914 localtime is 3:20 
finished day reading for group 3 of 10, time elapsed is 1237.869941 localtime is 3:20 


In [32]:
table.shape

(4200750, 6)

In [33]:
%%time
table['TIMESTAMP'] = table['DATE','TIME'].apply(lambda x: x['DATE'][0:4]+'-'+x['DATE'][4:6]+'-'+x['DATE'][6:8]
                                                +'-'+x['TIME'][0:2]+'-'+x['TIME'][2:4]+'-'+x['TIME'][4:6])

table['TIMESTAMP'] = table['TIMESTAMP'].str_to_datetime(str_format='%Y-%m-%d-%H:%M:%S')

#limpio la antenna hash, sacandome el ultimo caracter pues este no identifica la antenna sino que modifica el tipo de
#llamado utilizado en esa call solamente
#dropeo todas las celdas 
table =  table[table['CLIENT_CELL_ID'].apply(lambda x: len(x)>=5)]
table['CLIENT_CELL_ID'] = table['CLIENT_CELL_ID'].apply(lambda x: x[:-1] if len(x) ==6 else x)

table.remove_columns(['DATE','TIME'])

## leo si las llamadas fueron hechas durante el findesemana 
table['DURING_WEEKEND'] = table['TIMESTAMP'].apply(lambda x: x.weekday()==5 or x.weekday()==6)

# asigno el bool si la llamada fue en horario 'laboral' o DIURNO (cuando hay sol)
#notar que la hora < 20 significa que 19hs y 59 min da True
table['DURING_DAY']= table['TIMESTAMP'].apply(lambda x: x.hour>=5 and x.hour<20 )

CPU times: user 1min 5s, sys: 6.39 s, total: 1min 12s
Wall time: 15.6 s


In [93]:
%%time
#me quedo con este pequenyo corte de la tabla
table_weeknight = table[table['DURING_WEEKEND']==False and table['DURING_DAY']==False ]

#hago el primer procesamiento aca
table_weeknight = table_weeknight.groupby(['USER','CLIENT_CELL_ID'],
                                                    {'ANTENNA_COUNT_WEEKNIGHT':gl.aggregate.COUNT()})

table_weeknight = table_weeknight.groupby(['USER'],                                             {'ANTENNA_COUNT_DICT_WEEKNIGHT':gl.aggregate.CONCAT("CLIENT_CELL_ID","ANTENNA_COUNT_WEEKNIGHT")})

table_weeknight['ANTENNA_ID_WEEKNIGHT'] = table_weeknight['ANTENNA_COUNT_DICT_WEEKNIGHT'].\
apply(lambda user: get_columns_from_dict(user))
table_weeknight['COUNT_WEEKNIGHT'] = table_weeknight['ANTENNA_ID_WEEKNIGHT'].\
apply(lambda row: row[1])
table_weeknight['ANTENNA_ID_WEEKNIGHT'] = table_weeknight['ANTENNA_ID_WEEKNIGHT'].\
apply(lambda row: row[0])
table_weeknight = table_weeknight.unpack('ANTENNA_ID_WEEKNIGHT') 
table_weeknight = table_weeknight.unpack('COUNT_WEEKNIGHT') 
table_weeknight = table_weeknight.\
rename(dict([(col,col.replace(".","_")) for col in table_weeknight.column_names() if ("." in col)]))

## falta dropear el column dict nomas
table_weeknight = table_weeknight.remove_column('ANTENNA_COUNT_DICT_WEEKNIGHT')

CPU times: user 23.4 s, sys: 4.1 s, total: 27.5 s
Wall time: 12.2 s


In [ ]:
#chequeo por las dudas que no este levantando antennas invisibles

In [34]:
test = table.filter_by(["CTI " + str(i) for i in range(1,10)], 'CLIENT_CELL_ID')
print(table.shape, test.shape)
print(table[table['CLIENT_CELL_ID']=='CTI 1'].shape)



## Problema
existen antennas tipo _CTI 1_ , _CTI 2_ , _CTI 3_ ,... que NO podemos encontrar en los dataset de antennas. Probamos con el dataset mas crudo llamado `celdas.csv` a ver si existen.

In [127]:
antennas_raw_file = '/home/juan/mobility-study/argentina-scripts/data/celdas.csv'
!head $antennas_raw_file

CEL_ID,CEL_DESCRIPTION,LATITUD,LONGITUD
CO534D,CHATEAU - VILLA AGUR,-31.3703,-64.2403
CO534F,CHATEAU - VILLA AGUR,-31.3703,-64.2403
CO534X,CHATEAU - VILLA AGUR,-31.3703,-64.2403
CO535A,BARRIO EL ROSEDAL,-31.4353,-64.2248
CO535B,BARRIO EL ROSEDAL,-31.4353,-64.2248
CO535C,BARRIO EL ROSEDAL,-31.4353,-64.2248
CO535D,BARRIO EL ROSEDAL,-31.4353,-64.2248
CO535E,BARRIO EL ROSEDAL,-31.4353,-64.2248
CO535X,BARRIO EL ROSEDAL,-31.4353,-64.2248


In [128]:
antennas_raw = gl.SFrame.read_csv( antennas_raw_file, delimiter=',', 
                                         header=True, skip_initial_space=True, 
                                         column_type_hints = [str,str,float, float], 
                                         na_values=['NaN'],
                                         usecols = ['CEL_ID','CEL_DESCRIPTION','LATITUD',
                                                    'LONGITUD'],  
                                         error_bad_lines=False,
                                         verbose = False
                                        )


In [131]:
antennas_raw.filter_by(["CTI " + str(i) for i in range(1,10)], 'CEL_ID')

CEL_ID,CEL_DESCRIPTION,LATITUD,LONGITUD


In [107]:
table_weeknight.sample(0.001)

USER,ANTENNA_ID_WEEKNIGHT_0,ANTENNA_ID_WEEKNIGHT_1,ANTENNA_ID_WEEKNIGHT_2,ANTENNA_ID_WEEKNIGHT_3
6B5942786C376A595A766869744262764B2F4F4958773D3D ...,CTI 1,-1,-1,-1
456443396F4355306F33424331373054514C687637773D3D ...,CO224,CO133,CO214,CO273
6D48496A6E4E5A47656D5A52666F4D59712F735052513D3D ...,SF932,SF964,SF815,-1
72397974764F344C77767356444156725948537571673D3D ...,CTI 2,-1,-1,-1
65513831675976376E433064567279586E72684D32773D3D ...,CTI 2,-1,-1,-1
5447764B7A786E375353316B4D43364E5769634B74513D3D ...,CO684,-1,-1,-1
584546704373325373414D6D4472693635724A7734413D3D ...,CTI 2,-1,-1,-1
68444B6453652F444D45337A6E6E66505A464E3243673D3D ...,C1890,-1,-1,-1
7046524C3961632F4F2B464945522F3076415A476F673D3D ...,CTI 1,-1,-1,-1
687746537A42715468756466717374443143313046673D3D ...,CTI 2,-1,-1,-1


In [ ]:
#continuo con el resto del procesamiento, pero en la tabla sin filtrar
table = table.groupby(['USER','CLIENT_CELL_ID'],
                      {'ANTENNA_COUNT':gl.aggregate.COUNT()})

table = table.groupby(['USER'],
                     {'ANTENNA_COUNT_DICT':gl.aggregate.CONCAT("CLIENT_CELL_ID","ANTENNA_COUNT")})

table['ANTENNA_ID'] = table['ANTENNA_COUNT_DICT'].apply(lambda user: get_columns_from_dict(user))
table['COUNT'] = table['ANTENNA_ID'].apply(lambda row: row[1])
table['ANTENNA_ID'] = table['ANTENNA_ID'].apply(lambda row: row[0])
table = table.unpack('ANTENNA_ID') 
table = table.unpack('COUNT') 
table = table.rename(dict([(col,col.replace(".","_")) for col in table.column_names() if ("." in col)]))

## falta dropear el column dict nomas
table = table.remove_column('ANTENNA_COUNT_DICT')

#juntamos las dos tablas procesadas
table = table.join(table_weeknight,on = 'USER',how = 'left')

del table_weeknight

#relleno todos los NANs que van a aparecer
for col in [col for col in table.column_names() if ("ANTENNA" in col or "COUNT" in col )]:
    if "ANTENNA" in col:
        table = table.fillna(col,value="-1")
        if "COUNT" in col:
            table = table.fillna(col,value=-1)
            print('finished data processing from group {g} of {p}, time elapsed is {t}\n Start saving output files '.format(g = group, p = passes,t=(time.time()-start_time)))


print("Finished saving group {gr} of {p}, time elapsed is {t}\n ".format(gr=group,p=passes,t=(time.time()-start_time)))

then = start_time
seconds = time.time() - then

In [64]:
sfr  = "/home/juan/mobility-study/argentina-scripts/output/user_antenna_mapping_sframe_group0_1/" 
!du -sh $sfr

125M	/home/juan/mobility-study/argentina-scripts/output/user_antenna_mapping_sframe_group0_1/


In [65]:
test = gl.load_sframe(sfr)

In [71]:
[col for col in test.column_names() if ("ANTENNA" in col or "COUNT" in col )]

['ANTENNA_ID_0',
 'ANTENNA_ID_1',
 'ANTENNA_ID_2',
 'ANTENNA_ID_3',
 'ANTENNA_ID_4',
 'ANTENNA_ID_5',
 'ANTENNA_ID_6',
 'ANTENNA_ID_7',
 'ANTENNA_ID_8',
 'ANTENNA_ID_9',
 'COUNT_0',
 'COUNT_1',
 'COUNT_2',
 'COUNT_3',
 'COUNT_4',
 'COUNT_5',
 'COUNT_6',
 'COUNT_7',
 'COUNT_8',
 'COUNT_9',
 'ANTENNA_ID_NO_WORK_FRIDAY_0',
 'ANTENNA_ID_NO_WORK_FRIDAY_1',
 'ANTENNA_ID_NO_WORK_FRIDAY_2',
 'ANTENNA_ID_NO_WORK_FRIDAY_3',
 'ANTENNA_ID_NO_WORK_FRIDAY_4',
 'ANTENNA_ID_NO_WORK_FRIDAY_5',
 'ANTENNA_ID_NO_WORK_FRIDAY_6',
 'ANTENNA_ID_NO_WORK_FRIDAY_7',
 'ANTENNA_ID_NO_WORK_FRIDAY_8',
 'ANTENNA_ID_NO_WORK_FRIDAY_9',
 'COUNT_NO_WORK_FRIDAY_0',
 'COUNT_NO_WORK_FRIDAY_1',
 'COUNT_NO_WORK_FRIDAY_2',
 'COUNT_NO_WORK_FRIDAY_3',
 'COUNT_NO_WORK_FRIDAY_4',
 'COUNT_NO_WORK_FRIDAY_5',
 'COUNT_NO_WORK_FRIDAY_6',
 'COUNT_NO_WORK_FRIDAY_7',
 'COUNT_NO_WORK_FRIDAY_8',
 'COUNT_NO_WORK_FRIDAY_9']

## cambiamos el working dir y el input_file (output.txt.gz --> output1.txt.gz)
Esto pues vamos salvando los resultados del procesamiento de datos en files sucesivos para no tener que repetir el algoritmo ni para arriesgar cambiar una version (en terminos de datos) y no tener checkpoints intermedios

In [10]:
#ejemplo  "/grandata/simple_format/01-02012014.txt.gz"
#seteamos el lugar de trabajo
rootdir="/home/juan/mobility-study/"
os.chdir(rootdir)
year = "2015"; 
month_start= "01";
#todos estos sum_links terminan 2 meses mas tardes
month_end= str(int(month_start)+2)
if int(month_end )<10:
    month_end="0"+month_end
    
day_start = "01"
day_end = str(int(day_start)+1)
if int(day_end )<10:
    day_end="0"+ day_end

input_file= rootdir +"output1.txt.gz"\
                .format(y=year,ms=month_start,me=month_end,
                       ds=day_start,de=day_end)
output_file = "/home/juan/mobility-study/output2"
if week_end == True:
    output_file = output_file + "_wkend"
if night_filter == True:
    output_file += "_ngtfilter"
if hash_map == True:
    output_file += "_user_hash_map"
output_file += ".txt.gz"

In [11]:
!echo $input_file

/home/juan/mobility-study/output1.txt.gz


In [12]:
!du -ha $input_file

165M	/home/juan/mobility-study/output1.txt.gz


In [13]:
!zcat $input_file | head -n20

USER,EXPOSED,ANTENNA_ID_0,count_0,ANTENNA_ID_1,count_1,ANTENNA_ID_2,count_2,ANTENNA_ID_3,count_3,ANTENNA_ID_4,count_4,ANTENNA_ID_5,count_5,ANTENNA_ID_6,count_6,ANTENNA_ID_7,count_7,ANTENNA_ID_8,count_8,ANTENNA_ID_9,count_9,EPIDEMIC
FFFF78CB080636632B2D1DE7A7BFAB03,0,3415,21,2969,5,1320,3,2471,3,1115,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
FFFF51FDDD55A2E24D074DE30C4798E9,1,1207,15,1171,14,1820,9,583,8,2026,5,1066,4,1831,4,1625,2,401,1,825,1,0
FFFE1EDDA25369CBC467536FA6A787FD,1,1847,22,1753,14,1422,11,1028,10,765,9,3072,9,862,8,2463,7,1416,6,871,5,1
FFFD0A912A959CDFCFFAB2F93BFBA435,1,1407,18,1702,8,1613,6,1704,5,1394,3,1398,3,818,2,958,2,726,1,848,1,0
FFFCB235D6D917DDF350F5B11CCB0EE8,1,17,74,174,33,1123,4,1178,4,173,3,890,2,503,1,1039,1,1228,1,1327,1,1
FFFC9EC4F662C44D3006BA69C9F0DE85,1,876,19,1737,6,1744,5,326,1,378,1,503,1,1118,1,2216,1,2477,1,2962,1,1
FFFB2F18F4CCF253C06B581EA518A7A5,1,2543,13,704,7,2540,7,1453,6,1642,5,45,4,2393,4,1841,3,3031,3,1009,2,0
FFFA147CB2B23AD12F207F0EFBEB14D0,1,

In [14]:
table = pd.read_csv(
                input_file,
                engine = 'c',
                sep = ',',
                header = 0,
                index_col=0,
                dtype = {'EXPOSED':np.bool,'EPIDEMIC':np.bool,
                         'ANTENNA_ID_0':np.int16,'count_0':np.uint16,'ANTENNA_ID_1':np.int16,'count_1':np.uint16,
                         'ANTENNA_ID_2':np.int16,'count_2':np.uint16,'ANTENNA_ID_3':np.int16,'count_3':np.uint16,
                        'ANTENNA_ID_4':np.int16,'count_4':np.uint16,'ANTENNA_ID_5':np.int16,'count_5':np.uint16,
                        'ANTENNA_ID_6':np.int16,'count_6':np.uint16,'ANTENNA_ID_7':np.int16,'count_7':np.uint16,
                        'ANTENNA_ID_8':np.int16,'count_8':np.uint16,'ANTENNA_ID_9':np.int16,'count_9':np.uint16}
                )
## ojo con setear los dtypes para las antennas en np.usigned int pues ahi me transforma todos los -1s en el max del dtype

In [15]:
col_labels = [ col.replace("count","COUNT") if ("count" in col) else col for col in table.columns]
table.columns = col_labels

In [16]:
#df['new_col'] = zip(df.lat, df.long)
antenna_lat_long_dict = dict(zip(antennas.index.values, zip(antennas.LATITUDE,antennas.LONGITUDE)))

In [17]:
import math
#earht_radi in kms
earth_radi = 6371

def distance_on_unit_sphere(lat1, long1, lat2, long2, earth_radi=6371):
 
    # Convert latitude and longitude to 
    # spherical coordinates in radians.
    degrees_to_radians = math.pi/180.0
         
    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians
         
    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians
         
    # Compute spherical distance from spherical coordinates.
         
    # For two locations in spherical coordinates 
    # (1, theta, phi) and (1, theta', phi')
    # cosine( arc length ) = 
    #    sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length
     
    cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) + 
           math.cos(phi1)*math.cos(phi2))
    arc = math.acos( cos )
 
    # Remember to multiply arc by the radius of the earth 
    # in your favorite set of units to get length.
    return arc*earth_radi

In [18]:
from scipy import spatial

### checking antenna_id consistency
para ver si todos los antenna_ids de nuestra tabla estan en la base de datos de las antennas.

In [17]:
def table_antennaid_consistency(row,antenna_id_list):
    res = False
    col_labels = table.columns
    #miramos solo las columnas de antenna_ids 
    row = row[[col for col in table.columns if ("ANTENNA" in col)]]
    #filtramos aquellas que tienen ant_id no nulo (==-1)
    row = row[row != "-1"]
    #print(row)
    #le pregunto a la serie si c/valor esta en el id_list, paso a un ndarray y consulto si todos estan
    res = row.isin(antenna_id_list).values.all()
    
    return res

In [20]:
antenna_id_list = list(antennas.index.values)

In [21]:
%%time
consistency = table.sample(10**5).apply(func = (lambda row: table_antennaid_consistency(row,antenna_id_list)),axis='columns')
#tableeo que todos los resultados sean True
print(consistency.values.all())
del consistency


False
CPU times: user 2min 18s, sys: 76 ms, total: 2min 18s
Wall time: 2min 18s


In [141]:
user = 'FFFA147CB2B23AD12F207F0EFBEB14D0'
row = table.loc[user]
row = row[row>-1]
row = row[[ind for ind in row.index if "ANTENNA" in ind]]

### END antenna consistency table

In [22]:
##algoritmo sin convex hull

filter_col = [col for col in list(table) if col.startswith('ANTENNA')]

def convex_hull_radi(row,antenna_lat_long_dict,filter_col):
    #rows are pandas series
    #print(row.name)
    row =  row[filter_col]
    latitudes=[]
    longitudes =[]
    
    #cuidado con los NaNs resultantes del map
    for ant_id in (row.map(antenna_lat_long_dict).dropna()):
        latitudes += [ant_id[0]] 
        longitudes += [ant_id[1]]
    
    
    #try:
    radii=0
    for i in range(len(latitudes)):
        for j in range(i):
            new_dist= distance_on_unit_sphere(latitudes[i], longitudes[i], latitudes[j], longitudes[j])
            if radii< new_dist:
                radii = new_dist 

    return radii
    #except:
    #    print(row.name)
    #    return 0
        
       
#row =table_table.loc[2][filter_col]
#latitudes=[]
#longitudes =[]
#for ant_id in row.map(antenna_lat_long_dict).dropna():
#    latitudes += [ant_id[0]] 
#    longitudes += [ant_id[1]]
#latitudes

#convex_hull_radi(table_table.loc[12],antenna_lat_long_dict,filter_col)

In [23]:
## algoritmo con convexhulll

filter_col = [col for col in list(table) if col.startswith('ANTENNA')]

def convex_hull_radi(row,antenna_lat_long_dict,filter_col):
    #rows are pandas series
    #print(row.name)
    row =  row[filter_col]
    latitudes=[]
    longitudes =[]
    
    #cuidado con los NaNs resultantes del map
    for ant_id in (row.map(antenna_lat_long_dict).dropna()):
        latitudes += [ant_id[0]] 
        longitudes += [ant_id[1]]
    #case: only 1 antennas
    if len(latitudes)<=1:
        return 0
    
  
    
    #case only 2 antennas, arc distance
    elif len(latitudes)==2:
        return distance_on_unit_sphere(latitudes[0],longitudes[0],latitudes[1],longitudes[1])
    
    
    elif  len(latitudes)>2:
        points = np.array([np.asarray(latitudes).transpose() ,np.asarray(longitudes).transpose()])
        points = points.transpose()
        
        #chequeamos los casos super patologicos que todas las antenas esten sobre la misma linea
        if latitudes.count(latitudes[0]) == len(latitudes):
            radii=0

            for i in range(len(longitudes)-1):
                new_dist= distance_on_unit_sphere(latitudes[0], longitudes[i], latitudes[0], longitudes[i+1])
                if radii< new_dist:
                    radii = new_dist 
        
        #chequear sobre la otra linea
        elif longitudes.count(longitudes[0]) == len(longitudes):    
            radii=0
            for i in range(len(latitudes)-1):
                new_dist= distance_on_unit_sphere(latitudes[i], longitudes[0], latitudes[i+1], longitudes[0])
                if radii< new_dist:
                    radii = new_dist 
        else:
            try:
                chull = spatial.ConvexHull(points)
                #points = spatial.ConvexHull(points).points
                points = chull.points
            except:
                print(row.name)
            vert = [list(point) for point in chull.points] #chull vertices

            radii=0
            for i in range(len(vert)):
                for j in range(i):
                    new_dist= distance_on_unit_sphere(vert[i][0], vert[i][1], vert[j][0], vert[j][1])
                    if radii< new_dist:
                        radii = new_dist 

        return radii
        #return ("%s,len(lats) is %d" %(radii,len(latitudes)))

#convex_hull_radi(table_table.loc[12],antenna_lat_long_dict,filter_col)

In [ ]:
start_time = time.time()
#mini_sample =  table.sample(10**3)
table['MOBILITY_DIAMETER']= table.apply(lambda row: convex_hull_radi(row,antenna_lat_long_dict,filter_col),axis=1)

print("elapsed time is %s seconds for %d amount of rows" % (time.time()-start_time,len(mini_sample)))

In [309]:
antennas.loc[1695].LONGITUDE, antennas.loc[1311].LONGITUDE,antennas.loc[1179].LONGITUDE

(-99.191000000000003, -99.191000000000003, -99.191000000000003)

In [317]:
table.columns

Index([u'EXPOSED', u'ANTENNA_ID_0', u'COUNT_0', u'ANTENNA_ID_1', u'COUNT_1',
       u'ANTENNA_ID_2', u'COUNT_2', u'ANTENNA_ID_3', u'COUNT_3',
       u'ANTENNA_ID_4', u'COUNT_4', u'ANTENNA_ID_5', u'COUNT_5',
       u'ANTENNA_ID_6', u'COUNT_6', u'ANTENNA_ID_7', u'COUNT_7',
       u'ANTENNA_ID_8', u'COUNT_8', u'ANTENNA_ID_9', u'COUNT_9', u'EPIDEMIC',
       u'MOBILITY_DIAMETER'],
      dtype='object')

## un par de corridas usando el diametro del convex/hull dan 
elapsed time is 253.765459061 for 40000 amount of rows

elapsed time is 249.712596893 seconds for 40000 amount of rows

elapsed time is 227.963900089 seconds for 40000 amount of rows

elapsed time is 57.2980730534 seconds for 10000 amount of rows

elapsed time is 0.896952867508 seconds for 10 amount of rows



### tableeamos la validez de resultados
Nos fijamos en todos los users que tengan al menos dos antenas, en los users que solo tienen 1 antenna, etc.

In [249]:
mini_sample[(mini_sample['MOBILITY_DIAMETER']<10**3) & (mini_sample['ANTENNA_ID_0']>-1)].shape


(312, 23)

In [25]:
table.head()

,EXPOSED,ANTENNA_ID_0,COUNT_0,ANTENNA_ID_1,COUNT_1,ANTENNA_ID_2,COUNT_2,ANTENNA_ID_3,COUNT_3,ANTENNA_ID_4,...,COUNT_5,ANTENNA_ID_6,COUNT_6,ANTENNA_ID_7,COUNT_7,ANTENNA_ID_8,COUNT_8,ANTENNA_ID_9,COUNT_9,EPIDEMIC
USER,,,,,,,,,,,,,,,,,,,,,
FFFF78CB080636632B2D1DE7A7BFAB03,False,3415,21,2969,5,1320,3,2471,3,1115,...,65535,-1,65535,-1,65535,-1,65535,-1,65535,False
FFFF51FDDD55A2E24D074DE30C4798E9,True,1207,15,1171,14,1820,9,583,8,2026,...,4,1831,4,1625,2,401,1,825,1,False
FFFE1EDDA25369CBC467536FA6A787FD,True,1847,22,1753,14,1422,11,1028,10,765,...,9,862,8,2463,7,1416,6,871,5,True
FFFD0A912A959CDFCFFAB2F93BFBA435,True,1407,18,1702,8,1613,6,1704,5,1394,...,3,818,2,958,2,726,1,848,1,False
FFFCB235D6D917DDF350F5B11CCB0EE8,True,17,74,174,33,1123,4,1178,4,173,...,2,503,1,1039,1,1228,1,1327,1,True


In [27]:
len(np.unique(table.index.values)), table.shape

(3667451, (3667451, 22))

In [264]:
#mob_table = mini_sample['mobility_diameter']
results = mini_sample[(mini_sample['MOBILITY_DIAMETER']<10**3) & (mini_sample['ANTENNA_ID_8']>-1)].head()
print(results.shape)
for user in results.index:
    row = table.loc[user]
    row = row[row>-1]
    row = row[[ind for ind in row.index if "ANTENNA" in ind]]
    print(row)

(5, 23)
ANTENNA_ID_0    1149
ANTENNA_ID_1    1573
ANTENNA_ID_2    2812
ANTENNA_ID_3    2948
ANTENNA_ID_4      82
ANTENNA_ID_5     189
ANTENNA_ID_6     233
ANTENNA_ID_7     713
ANTENNA_ID_8    1477
ANTENNA_ID_9    1493
Name: 5D51048D591291F351EF09EADBD12F53, dtype: object
ANTENNA_ID_0      29
ANTENNA_ID_1    1700
ANTENNA_ID_2      73
ANTENNA_ID_3    2347
ANTENNA_ID_4      72
ANTENNA_ID_5    1235
ANTENNA_ID_6    1450
ANTENNA_ID_7    1561
ANTENNA_ID_8     674
ANTENNA_ID_9    1290
Name: 7BC884A1A97BEA2573B0BE18F7DB085C, dtype: object
ANTENNA_ID_0    1637
ANTENNA_ID_1    1352
ANTENNA_ID_2    1176
ANTENNA_ID_3    2026
ANTENNA_ID_4     268
ANTENNA_ID_5     942
ANTENNA_ID_6    1167
ANTENNA_ID_7    1255
ANTENNA_ID_8    1324
ANTENNA_ID_9    2968
Name: DD1ED3EB5C1AD91A7D590C5E1841F684, dtype: object
ANTENNA_ID_0    2043
ANTENNA_ID_1    1080
ANTENNA_ID_2    2323
ANTENNA_ID_3    2561
ANTENNA_ID_4    2269
ANTENNA_ID_5    2268
ANTENNA_ID_6    1097
ANTENNA_ID_7    1979
ANTENNA_ID_8    3152
ANTENNA_ID_

In [265]:
#tableeamos que todos los users con 1 antenna tengan result ==0 
mob_table = mini_sample['MOBILITY_DIAMETER']
results = mini_sample[(mini_sample['MOBILITY_DIAMETER']<10) & (mini_sample['ANTENNA_ID_1']==-1)]
#mob_table[(mob_table.values<10) & (mini_sample['ANTENNA_ID_1']==-1)]
(results['MOBILITY_DIAMETER'].values==0).all()


True

### End mobility diameter result tables, Save table

In [269]:
mini_sample[[col for col in mini_sample.columns if ("ANTENNA" in col or "MOBILITY" in col)]].sample(10)

,ANTENNA_ID_0,ANTENNA_ID_1,ANTENNA_ID_2,ANTENNA_ID_3,ANTENNA_ID_4,ANTENNA_ID_5,ANTENNA_ID_6,ANTENNA_ID_7,ANTENNA_ID_8,ANTENNA_ID_9,MOBILITY_DIAMETER
USER,,,,,,,,,,,
9AF7A02045CB182FE90A88C9D03130AB,2178,392,2010,709,1407,1412,2084,520,2041,1083,1368.713640
45AEB0A267D2BFFD3983EF2C95CA206B,2964,2943,2301,3673,1012,3790,634,3681,1463,2404,2325.646079
3243E73992D14766D85D1F282F4C7690,291,2596,561,655,887,3143,3577,-1,-1,-1,1594.044422
32F7FFA33F10A88CEDEFE8D9BD4060F0,20,22,31,-1,-1,-1,-1,-1,-1,-1,707.770266
2A50CA984894ADD3989C8920D4B479AE,400,161,2322,1216,1326,1292,1565,637,1088,1205,1716.188192
A81BC0F14993A91B60E780236F7EBB0E,2175,780,911,1820,1766,1991,3199,1043,1649,1938,1534.150943
B8C676F2420B18EB3794FD4E8CAFC2BA,1247,1244,564,1697,307,2788,197,502,609,1171,1549.084125
168F7B598F3AD64679C123DFAC59D9B2,2746,218,-1,-1,-1,-1,-1,-1,-1,-1,524.782360
96397BB4600DE9D8440E0F1A77E30740,1637,1758,1438,1990,1324,1385,1750,1429,1601,1621,1447.386187


In [318]:
table.to_csv(output_file, 
             index = True,
            header = True,
            compression='gzip')

In [19]:
!du -ha $output_file*

173M	/home/juan/mobility-study/output2.txt.gz


In [323]:
table.sample(20)

,EXPOSED,ANTENNA_ID_0,COUNT_0,ANTENNA_ID_1,COUNT_1,ANTENNA_ID_2,COUNT_2,ANTENNA_ID_3,COUNT_3,ANTENNA_ID_4,...,ANTENNA_ID_6,COUNT_6,ANTENNA_ID_7,COUNT_7,ANTENNA_ID_8,COUNT_8,ANTENNA_ID_9,COUNT_9,EPIDEMIC,MOBILITY_DIAMETER
USER,,,,,,,,,,,,,,,,,,,,,
18113EA238478375460839F4F5F3B017,True,1083,10,798,5,1608,5,248,4,849,...,2679,3,369,2,1016,1,1245,1,True,1596.220319
4C21FDD80CD47E153298C70D3527F079,True,3183,37,2875,31,2298,26,1449,16,2299,...,2296,4,2745,4,1154,3,1258,2,False,2847.255740
5B69FC601B10AD78AF0E250899DA4428,True,989,8,1379,8,327,7,990,6,1431,...,1610,3,988,1,1074,1,1098,1,True,737.250275
3104F21B0C633119F9EB28433F5DF0AC,True,2277,13,1990,5,1758,2,-1,65535,-1,...,-1,65535,-1,65535,-1,65535,-1,65535,True,739.327422
794E3583897B762E37DB26CB29BF2C95,True,1461,27,3825,19,2895,18,1012,17,2584,...,2055,6,2275,6,3787,4,2043,3,True,2655.544887
D3B4C0718A0A4B8B3A0495B9256C1262,True,540,4,1315,4,2013,3,2014,3,763,...,2520,2,-1,65535,-1,65535,-1,65535,False,701.064183
D0085B84A7EB85956E996C0EF2201453,True,594,43,2276,27,515,20,1386,17,2316,...,2174,13,771,12,1319,10,1721,10,True,742.937402
B10EEF8023AA67CD510AF3F9BD0D1405,True,399,33,1508,20,1415,11,526,10,1483,...,873,6,1573,5,111,4,1356,4,False,559.340031
3ECB89A445B6C708219724E905EF17D6,True,197,9,2362,8,2084,4,2278,4,2197,...,391,1,484,1,572,1,709,1,False,1368.713640


##  In case of antenna epidemic tag change
En el case en que cambien la definicion de la zona epidemica, esto cambiaria la condicion de epidemicidad para c/ antenna

In [8]:
#seteamos el lugar de trabajo
rootdir="/home/juan/mobility-study/"
os.chdir(rootdir)
year = "2015"; 

input_file= rootdir +"output2.txt.gz"\
                .format(y=year,ms=month_start,me=month_end,
                       ds=day_start,de=day_end)
    
output_file = "/home/juan/mobility-study/output2"
if week_end == True:
    output_file = output_file + "_wkend"
if night_filter == True:
    output_file += "_ngtfilter"
if hash_map == True:
    output_file += "_user_hash_map"
output_file += ".txt.gz"

In [9]:
!zcat $input_file | head -n10

USER,ANTENNA_ID_0,COUNT_0,ANTENNA_ID_1,COUNT_1,ANTENNA_ID_2,COUNT_2,ANTENNA_ID_3,COUNT_3,ANTENNA_ID_4,COUNT_4,ANTENNA_ID_5,COUNT_5,ANTENNA_ID_6,COUNT_6,ANTENNA_ID_7,COUNT_7,ANTENNA_ID_8,COUNT_8,ANTENNA_ID_9,COUNT_9,MOBILITY_DIAMETER,EXPOSED,EPIDEMIC
FFFF78CB080636632B2D1DE7A7BFAB03,3415,21,2969,5,1320,3,2471,3,1115,2,-1,65535,-1,65535,-1,65535,-1,65535,-1,65535,1063,0,0
FFFF51FDDD55A2E24D074DE30C4798E9,1207,15,1171,14,1820,9,583,8,2026,5,1066,4,1831,4,1625,2,401,1,825,1,520,1,0
FFFE1EDDA25369CBC467536FA6A787FD,1847,22,1753,14,1422,11,1028,10,765,9,3072,9,862,8,2463,7,1416,6,871,5,990,1,0
FFFD0A912A959CDFCFFAB2F93BFBA435,1407,18,1702,8,1613,6,1704,5,1394,3,1398,3,818,2,958,2,726,1,848,1,491,1,0
FFFCB235D6D917DDF350F5B11CCB0EE8,17,74,174,33,1123,4,1178,4,173,3,890,2,503,1,1039,1,1228,1,1327,1,1613,1,0
FFFC9EC4F662C44D3006BA69C9F0DE85,876,19,1737,6,1744,5,326,1,378,1,503,1,1118,1,2216,1,2477,1,2962,1,3004,1,1
FFFB2F18F4CCF253C06B581EA518A7A5,2543,13,704,7,2540,7,1453,6,1642,5,45,4,2393,4,

In [10]:
#primero leo el file de antennas
antennas= pd.read_csv('/home/juan/mobility-study/antennas_mexico.csv',sep = "|",header=0,index_col=0)
antennas.index.rename("ANTENNA_ID",inplace=True)

In [11]:
table = pd.read_csv(
                input_file,
                engine = 'c',
                sep = '|',
                header = 0,
                index_col=0,
                dtype = {'EXPOSED':np.bool,'EPIDEMIC':np.bool, 'MOBILITY_DIAMETER':np.float32,
                         'ANTENNA_ID_0':np.int16,'COUNT_0':np.uint16,'ANTENNA_ID_1':np.int16,'COUNT_1':np.uint16,
                         'ANTENNA_ID_2':np.int16,'COUNT_2':np.uint16,'ANTENNA_ID_3':np.int16,'COUNT_3':np.uint16,
                        'ANTENNA_ID_4':np.int16,'COUNT_4':np.uint16,'ANTENNA_ID_5':np.int16,'COUNT_5':np.uint16,
                        'ANTENNA_ID_6':np.int16,'COUNT_6':np.uint16,'ANTENNA_ID_7':np.int16,'COUNT_7':np.uint16,
                        'ANTENNA_ID_8':np.int16,'COUNT_8':np.uint16,'ANTENNA_ID_9':np.int16,'COUNT_9':np.uint16}
                )
table['MOBILITY_DIAMETER'] = table['MOBILITY_DIAMETER'].astype(int)
## ojo con setear los dtypes para las antennas en np.usigned int pues ahi me transforma todos los -1s en el max del dtype

In [ ]:
# elimino todas las columnas que quiero re-calcular
table.drop('EPIDEMIC',axis=1, inplace=True)
table.drop('EXPOSED',axis=1, inplace=True)

In [116]:
table.head()

,ANTENNA_ID_0,COUNT_0,ANTENNA_ID_1,COUNT_1,ANTENNA_ID_2,COUNT_2,ANTENNA_ID_3,COUNT_3,ANTENNA_ID_4,COUNT_4,...,COUNT_6,ANTENNA_ID_7,COUNT_7,ANTENNA_ID_8,COUNT_8,ANTENNA_ID_9,COUNT_9,MOBILITY_DIAMETER,EPIDEMIC,EXPOSED
USER,,,,,,,,,,,,,,,,,,,,,
FFFF78CB080636632B2D1DE7A7BFAB03,3415,21,2969,5,1320,3,2471,3,1115,2,...,65535,-1,65535,-1,65535,-1,65535,1063,False,True
FFFF51FDDD55A2E24D074DE30C4798E9,1207,15,1171,14,1820,9,583,8,2026,5,...,4,1625,2,401,1,825,1,520,False,True
FFFE1EDDA25369CBC467536FA6A787FD,1847,22,1753,14,1422,11,1028,10,765,9,...,8,2463,7,1416,6,871,5,990,False,True
FFFD0A912A959CDFCFFAB2F93BFBA435,1407,18,1702,8,1613,6,1704,5,1394,3,...,2,958,2,726,1,848,1,491,True,False
FFFCB235D6D917DDF350F5B11CCB0EE8,17,74,174,33,1123,4,1178,4,173,3,...,1,1039,1,1228,1,1327,1,1613,False,True


In [118]:
antennas.head()

,LATITUDE,LONGITUDE,STATE,EPIDEMIC
ANTENNA_ID_9,,,,
2797,14.671,-92.372,Chiapas,0
3060,14.682,-92.155,Chiapas,0
3344,14.721,-92.424,Chiapas,0
3487,14.778,-92.179,Chiapas,0
1924,14.786,-92.366,Chiapas,0


In [119]:
antena_ids = np.unique(antennas.index.values)
user_antenna_ids = set(np.unique(table['ANTENNA_ID_0'].values)) | set(np.unique(table['ANTENNA_ID_1'].values)) |\
                    set(np.unique(table['ANTENNA_ID_2'].values)) | set(np.unique(table['ANTENNA_ID_3'].values)) |\
                set(np.unique(table['ANTENNA_ID_4'].values)) | set(np.unique(table['ANTENNA_ID_5'].values)) |\
        set(np.unique(table['ANTENNA_ID_6'].values)) | set(np.unique(table['ANTENNA_ID_7'].values))
if -1 in user_antenna_ids:
    user_antenna_ids.remove(-1)
user_antenna_ids  = np.asarray(list(user_antenna_ids))

print(np.in1d(user_antenna_ids,antena_ids).all(), np.in1d(antena_ids,user_antenna_ids).all() )

print(np.in1d(antena_ids,user_antenna_ids).sum(), len(antena_ids))

(True, False)
(4221, 4429)


In [120]:
antenna_cols = [col for col in table.columns if "ANTENNA_ID" in col]

for col in antenna_cols:
    antennas.index.name = col
    table = table.join(antennas['EPIDEMIC'].astype(int), on=col,rsuffix='_{0}'.format(col.split("_")[-1]))
#solo falta renombrar la columna que no tiene rsuffix
table = table.rename(columns={'EPIDEMIC':'EPIDEMIC_0'})

In [128]:
epi_cols = [col for col in table.columns if 'EPIDEMIC_' in col]
table['EXPOSED'] = (table[epi_cols].sum(axis=1)>=1).astype(int)
table['EPIDEMIC'] =  table['EPIDEMIC_0'].values
for col in epi_cols:
    table.drop(col,axis=1,inplace=True)

In [129]:
table.sample(20)

,ANTENNA_ID_0,COUNT_0,ANTENNA_ID_1,COUNT_1,ANTENNA_ID_2,COUNT_2,ANTENNA_ID_3,COUNT_3,ANTENNA_ID_4,COUNT_4,...,COUNT_6,ANTENNA_ID_7,COUNT_7,ANTENNA_ID_8,COUNT_8,ANTENNA_ID_9,COUNT_9,MOBILITY_DIAMETER,EXPOSED,EPIDEMIC
USER,,,,,,,,,,,,,,,,,,,,,
ED96A156B5712B53C57CFF8E8ABC182D,2002,98,2204,40,1962,30,2353,23,1446,21,...,17,1443,15,1444,10,514,9,846,1,0
A53F04CCB9D4DE72D1D80A9695CA0F38,2040,57,2282,21,754,20,1279,14,1877,13,...,8,343,4,131,3,1328,3,1535,1,0
4921680006841C65C5960D6E9C31667F,2555,2,676,1,1002,1,3312,1,-1,65535,...,65535,-1,65535,-1,65535,-1,65535,2128,1,0
45690AA7F26E3FEF051AD1252E716CB8,125,13,1310,8,257,4,250,3,123,2,...,1,1018,1,1401,1,-1,65535,695,1,0
F1CBA36A18650AB67A10181F1232CB3D,2047,75,657,22,409,19,2359,19,256,18,...,16,2208,15,954,14,321,11,1007,1,0
FAF9615D48D09049AF12453F767E5B9E,1174,16,116,11,3053,7,2921,6,3054,5,...,3,2387,2,2716,2,3591,2,2850,1,1
A17D41FD01A77C21E2C73C85D3D6C865,2688,188,834,20,729,1,-1,65535,-1,65535,...,65535,-1,65535,-1,65535,-1,65535,912,0,0
84B9E30956F6A66675D17A15189F9CCB,2075,64,568,46,1821,36,1289,29,840,19,...,8,713,4,1960,4,2079,4,1056,1,0
6903CCA86378FDB0D1589C0D7FC3ACE3,566,5,276,4,945,4,1244,3,297,2,...,1,1125,1,1230,1,2082,1,959,1,0


In [130]:
#fast consistency check
(table['EXPOSED'] >= table['EPIDEMIC']).all()

True

In [151]:
table.drop_duplicates?

In [163]:
## individual samples check
n=1500
table = table[['ANTENNA_ID_0','EPIDEMIC']].sample(n).sort_values(by='ANTENNA_ID_0').drop_duplicates()
#el drop_duplicates es porque puede haber 2 usuarios con la misma antenna
table.head()


,ANTENNA_ID_0,EPIDEMIC
USER,,
69E679784251659B335C109AB9BDEE5C,1,0
1C74A9C234FAEC2953DD1E727DFBEB30,2,0
1689BA5694E0E9CA56325AE24721552A,6,0
08CC1463A441C353F8699BBDA7819BE9,8,1
5298821615BB5D5CF795BC6A5268924E,9,1


In [164]:
#comparo que la condicion epidemica sea igual para c/uno de los users una vez que sortie por las antenna_ids
#el sort_index es necesario pues son unicas las antennas
(antennas[np.in1d(antennas.index.values,table['ANTENNA_ID_0'].values)]['EPIDEMIC'].sort_index().values == 
 table.EPIDEMIC.values).all()

True

In [13]:
%%time
#save file back
table.to_csv(output_file,
             sep = "|",
             index = True,
            header = True,
            compression='gzip')

CPU times: user 1min 42s, sys: 352 ms, total: 1min 42s
Wall time: 1min 43s


In [172]:
#chequeo a ver como esta escrito
!zcat $output_file | head

USER,ANTENNA_ID_0,COUNT_0,ANTENNA_ID_1,COUNT_1,ANTENNA_ID_2,COUNT_2,ANTENNA_ID_3,COUNT_3,ANTENNA_ID_4,COUNT_4,ANTENNA_ID_5,COUNT_5,ANTENNA_ID_6,COUNT_6,ANTENNA_ID_7,COUNT_7,ANTENNA_ID_8,COUNT_8,ANTENNA_ID_9,COUNT_9,MOBILITY_DIAMETER,EXPOSED,EPIDEMIC
FFFF78CB080636632B2D1DE7A7BFAB03,3415,21,2969,5,1320,3,2471,3,1115,2,-1,65535,-1,65535,-1,65535,-1,65535,-1,65535,1063,0,0
FFFF51FDDD55A2E24D074DE30C4798E9,1207,15,1171,14,1820,9,583,8,2026,5,1066,4,1831,4,1625,2,401,1,825,1,520,1,0
FFFE1EDDA25369CBC467536FA6A787FD,1847,22,1753,14,1422,11,1028,10,765,9,3072,9,862,8,2463,7,1416,6,871,5,990,1,0
FFFD0A912A959CDFCFFAB2F93BFBA435,1407,18,1702,8,1613,6,1704,5,1394,3,1398,3,818,2,958,2,726,1,848,1,491,1,0
FFFCB235D6D917DDF350F5B11CCB0EE8,17,74,174,33,1123,4,1178,4,173,3,890,2,503,1,1039,1,1228,1,1327,1,1613,1,0
FFFC9EC4F662C44D3006BA69C9F0DE85,876,19,1737,6,1744,5,326,1,378,1,503,1,1118,1,2216,1,2477,1,2962,1,3004,1,1
FFFB2F18F4CCF253C06B581EA518A7A5,2543,13,704,7,2540,7,1453,6,1642,5,45,4,2393,4,